<h1>WFS1: Hildebrand, et al. (2008)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/18688868/" target="__blank">Hildebrand, et al. (2008) Autoimmune Disease in a DFNA6/14/38 Family Carrying a Novel Missense Mutation in WFS1</a></p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import numpy as np
from pyphetools.creation import *
from pyphetools.visualization import *
import pyphetools
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.6.4


<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
PMID = "PMID:18688868"
title = "Autoimmune disease in a DFNA6/14/38 family carrying a novel missense mutation in WFS1"
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", pmid=PMID, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('../../data/WFS1/PMID_18688868.xlsx')

In [4]:
df

,patient_id,II:2,III:1,III:3,IV:2,IV:4,V:2
0,Sex,female,female,female,female,female,male
1,Age,97,55,69,38,43,17
2,Variant,c.2576G>A,c.2576G>A,c.2576G>A,c.2576G>A,c.2576G>A,c.2576G>A
3,Low-frequency sensorineural hearing impairment,+,+,+,+,+,+
4,Progressive sensorineural hearing impairment,+,+,+,+,+,+
5,Graves disease,-,+,-,-,-,-
6,Crohn's disease,-,-,-,+,-,-


<h1>Converting to row-based format</h1>
<p>To use pyphetools, we need to have the individuals represented as rows (one row per individual) and have the items of interest be encoded as column names. The required transformations for doing this may be different for different input data, but often we will want to transpose the table (using the pandas <tt>transpose</tt> function) and set the column names of the new table to the zero-th row. After this, we drop the zero-th row (otherwise, it will be interpreted as an individual by the pyphetools code).</p>
<p>After this step is completed, the remaining steps to create phenopackets are the same as in the 
    <a href="http://localhost:8888/notebooks/notebooks/Create%20phenopackets%20from%20tabular%20data%20with%20individuals%20in%20rows.ipynb" target="__blank">row-based notebook</a>.</p>
    
Furthermore, for this specific case, there is a Count features row that we want dropped, so we filter out any row that does not have Patient in the first column.

In [5]:
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft.head()

patient_id,Sex,Age,Variant,Low-frequency sensorineural hearing impairment,Progressive sensorineural hearing impairment,Graves disease,Crohn's disease
II:2,female,97,c.2576G>A,+,+,-,-
III:1,female,55,c.2576G>A,+,+,+,-
III:3,female,69,c.2576G>A,+,+,-,-
IV:2,female,38,c.2576G>A,+,+,-,+
IV:4,female,43,c.2576G>A,+,+,-,-


Some column names might include spaces in front or after, and a couple of columns are subheadings and only contain NaNs, so lets correct that:

In [6]:
dft.columns = dft.columns.str.strip()
dft = dft.dropna(axis=1, how='all')
dft['patient_id'] = dft.index

<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [7]:
hpo_cr = parser.get_hpo_concept_recognizer()
generator = SimpleColumnMapperGenerator(df=dft, observed='+', excluded='-', hpo_cr=hpo_cr)
column_mapper_d = generator.try_mapping_columns()

In [8]:
from IPython.display import HTML, display
display(HTML(generator.to_html()))

Result,Columns
Mapped,Low-frequency sensorineural hearing impairment; Progressive sensorineural hearing impairment; Graves disease; Crohn's disease
Unmapped,Sex; Age; Variant; patient_id


<h2>Variant Data</h2>
<p>The variant data (HGVS transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [14]:
genome = 'hg38'
default_genotype = 'heterozygous'
WFS1_transcript='NM_006005.3'
var_list = dft['Variant'].unique()
vvalidator = VariantValidator(genome_build=genome, transcript=WFS1_transcript)
variant_d = {}
for v in var_list:
    var = vvalidator.encode_hgvs(v)
    variant_d[v] = var
    print(var)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3?content-type=application%2Fjson
chr4:6302371G>A


In [15]:
varMapper = VariantColumnMapper(variant_d=variant_d, variant_column_name='Variant', default_genotype=default_genotype)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [16]:
ageMapper = AgeColumnMapper.by_year('Age')
ageMapper.preview_column(dft['Age'])

,original column contents,age
0,97,P97Y
1,55,P55Y
2,69,P69Y
3,38,P38Y
4,43,P43Y
5,17,P17Y


In [17]:
sexMapper = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='Sex')
sexMapper.preview_column(dft['Sex'])

,original column contents,sex
0,female,FEMALE
1,female,FEMALE
2,female,FEMALE
3,female,FEMALE
4,female,FEMALE
5,male,MALE


In [19]:
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper, 
                        metadata=metadata,
                        pmid=PMID)
encoder.set_disease(disease_id='OMIM:600965', label='Deafness, autosomal dominant 6')

In [20]:
individuals = encoder.get_individuals()

In [22]:
from IPython.display import HTML, display

phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

In [23]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                             pmid=PMID,
                                             metadata=metadata.to_ga4gh(),
                                             outdir=output_directory)

We output 6 GA4GH phenopackets to the directory phenopackets
